Imports

In [1]:
import pandas as pd
import numpy as np

Constants

In [128]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'




VOCAB_SIZE = 2500

Read and Load Features from .txt files in to Numpy array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ' , dtype=int)

In [130]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)
sparse_test_data

array([[   8,    2,    1,    1],
       [   8,    3,    1,    2],
       [   8,    4,    1,    4],
       ...,
       [6891, 1952,    0,    1],
       [6891, 2077,    0,    1],
       [6891, 2104,    0,    2]])

In [6]:
sparse_train_data[-5:]

array([[6895, 2178,    0,    1],
       [6895, 2208,    0,    2],
       [6895, 2239,    0,    1],
       [6895, 2275,    0,    1],
       [6895, 2334,    0,    1]])

In [7]:
print('Nr of rows training file', sparse_train_data.shape[0])
print('Nr of rows testing file', sparse_test_data.shape[0])

Nr of rows training file 292923
Nr of rows testing file 129085


In [10]:
print('Nr of emails in training file' , np.unique(sparse_train_data[:,0]).size)
print('Nr of emails in testing file' , np.unique(sparse_test_data[:,0]).size)

Nr of emails in training file 4783
Nr of emails in testing file 2054


Create an Empty Dataframe

In [13]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0,VOCAB_SIZE))
len(column_names)

2502

In [23]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 6893, 6894, 6895])

In [21]:
full_train_data = pd.DataFrame(index=index_names , columns=column_names)
full_train_data.fillna(value=0, inplace=True)

C:\Users\jasee\AppData\Local\Temp\ipykernel_20576\227827799.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_train_data.fillna(value=0, inplace=True)


In [22]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Create a full matrix from a sparse matrix

In [25]:

#     form a full matrix from a sparse matrix , return a pandas dataframe
#     with doc_id, category, and uniques words in column
def make_full_matrix(sparse_matrix, nr_words ,doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3 ):
     
     column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0,VOCAB_SIZE))
     doc_id_names = np.unique(sparse_matrix[:,0])
     full_matrix = pd.DataFrame(index=doc_id_names , columns=column_names)
     full_matrix.fillna(value=0, inplace=True)
    
    
     for i in range(0, sparse_matrix.shape[0]):
          doc_nr = sparse_matrix[i][doc_idx]
          word_id = sparse_matrix[i][word_idx]
          label = sparse_matrix[i][cat_idx]
          occurence = sparse_matrix[i][freq_idx]
         
          full_matrix.at[doc_nr,'DOC_ID'] = doc_nr
          full_matrix.at[doc_nr,'CATEGORY'] = label
          full_matrix.at[doc_nr, word_id] = occurence
         
     full_matrix.set_index('DOC_ID' , inplace=True)

    
     return full_matrix
    

In [26]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

C:\Users\jasee\AppData\Local\Temp\ipykernel_20576\3506660413.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace=True)


CPU times: total: 17.7 s
Wall time: 17.7 s


In [28]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
6888,0,1,2,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6892,0,3,2,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6893,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6894,0,1,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
6895,0,1,2,1,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Training the Naives Bayes Model
Calculating the probability of spam

In [63]:
#Calculate the probability of spam - percentage of spam messages in the training datasets. store this value in a variable 
#called prob_spam

full_train_data.CATEGORY.size
full_train_data.loc[0:5,'CATEGORY']


DOC_ID
0    1
1    1
2    1
3    1
4    1
5    1
Name: CATEGORY, dtype: int64

In [42]:
full_train_data.CATEGORY.sum() # no of spam messages -> adding the all the ones

1248

In [174]:
Prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('probability of spam is ',Prob_spam)
ham_data = full_train_data[full_train_data.CATEGORY == 0] 
len(ham_data)
Prob_ham = len(ham_data) /  full_train_data.CATEGORY.size
Prob_ham 

probability of spam is  0.26092410620949197


0.7390758937905081

Total Number of Words/Tokens

In [51]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,1,2,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,4,2,3,0,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [59]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4783,)

In [64]:
email_lengths

DOC_ID
0        86
1        52
2        39
3       181
4        43
       ... 
6888     91
6892    108
6893     30
6894     63
6895     94
Length: 4783, dtype: int64

In [66]:
total_wc = email_lengths.sum()
total_wc

479386

Number of Tokens in Spam and Ham emails

In [92]:
# Create a subset of the email_length series that only contains the spam messages. call the subset spam_lenths.
#then count the total number of words that occur in spam emails,
# DO the same for non_spam emails. create ham_lenghts
email_lengths.shape
# subset_train_spam_data = full_train_data[full_train_data['CATEGORY'] == 1]
# full_spam_features = subset_train_spam_data.loc[:,full_train_data.columns != 'CATEGORY']
# spam_lengths = full_spam_features.sum(axis=1)
# # total no of spam words
# spam_wc = spam_lengths.sum()
# print('Nr of spam words are ', spam_wc)


# we do the same thing in this way 
spam_lengths = email_lengths[full_train_data['CATEGORY'] == 1]
spam_wc = spam_lengths.sum()
print('Nr of spam words are ', spam_wc)

Nr of spam words are  179261


In [90]:
#now of the Ham mails
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
nonspam_wc = ham_lengths.sum()
nonspam_wc.shape
print('Nr of ham words are ', nonspam_wc)
print('Nr of ham emails are ', ham_lengths.shape)

Nr of ham words are  300125
Nr of ham emails are  (3535,)


In [98]:
# Find the average no of words in spam and ham emails
print("Average nr of words in spam emails {:.0f}" .format(spam_wc / spam_lengths.shape[0]))
print("Average nr of words in ham emails {:.3f}" .format(nonspam_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 144
Average nr of words in ham emails 84.901


Summing the Tokens occuring in Spam

In [110]:
# following is subseting a Dataframe from another Dataframe
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,1,2,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,4,2,3,0,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [114]:
# to find how many times a word occurs in spam mail session
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 # 1 is added as part of laplace smooting inorder to avoid the zero probablity in likelihood function
summed_spam_tokens.head()

0    2195
1     944
2    1342
3    1298
4    1962
dtype: int64

In [115]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens.head()

0    6115
1    3055
2    1935
3    1886
4     953
dtype: int64

Calculate the Conditional Probablity
P(spam|word) = P(word|spam) * P(spam) /P(word)

P(Token|Spam) --> Probability that a Token Occurs given the Email is Spam

In [118]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam.sum()

1.0

P(Token|Ham) --> Probability that a Token Occurs given the Email is Ham

In [121]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0

P(Token) - Probability the Token Occurs

In [123]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all.sum()

0.9999999999999999

Save the Trained Model 

In [127]:
np.savetxt(TOKEN_SPAM_PROB_FILE , prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE , prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE , prob_tokens_all)

Prepare Test Data

In [134]:
%%time
full_test_data = make_full_matrix(sparse_test_data ,VOCAB_SIZE)

C:\Users\jasee\AppData\Local\Temp\ipykernel_20576\3506660413.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace=True)


CPU times: total: 7.7 s
Wall time: 7.7 s


In [146]:
X_test = full_test_data.loc[:,full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [147]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)